In [1403]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import display, clear_output
import json
import re
import websocket
import numpy as np
import typing
from scipy.stats import norm
import random
import time
import pandas as pd

In [1404]:
symbols = {
    "R_10": {
        "digits": 2,
        "precision": 3,
        "accu": {
            "barrier": {
                "0.01": 0.000061255198,
                "0.02": 0.000057252462,
                "0.03": 0.000053692776,
                "0.04": 0.000051086496,
                "0.05": 0.000048625396
            }
        }
    },
    "R_25": {
        "digits": 2,
        "precision": 3,
        "accu": {
            "barrier": {
                "0.01": 0.000153137996,
                "0.02": 0.000143131156,
                "0.03": 0.000134231941,
                "0.04": 0.00012771624,
                "0.05": 0.000121563489
            }
        }
    },
    "R_50": {
        "digits": 3,
        "precision": 4,
        "accu": {
            "barrier": {
                "0.01": 0.000306275997,
                "0.02": 0.000286262314,
                "0.03": 0.000268463882,
                "0.04": 0.00025543248,
                "0.05": 0.000243126978
            }
        }
    },
    "R_75": {
        "digits": 3,
        "precision": 4,
        "accu": {
            "barrier": {
                "0.01": 0.000459414004,
                "0.02": 0.000429393474,
                "0.03": 0.000402695823,
                "0.04": 0.000383148719,
                "0.05": 0.000364690464
            }
        }
    },
    "R_100": {
        "digits": 1,
        "precision": 2,
        "accu": {
            "barrier": {
                "0.01": 0.000612552024,
                "0.02": 0.000572524639,
                "0.03": 0.000536927765,
                "0.04": 0.000510864957,
                "0.05": 0.000486253948
            }
        }
    },
    "1HZ10V": {
        "digits": 1,
        "precision": 2,
        "accu": {
            "barrier": {
                "0.01": 0.000043313966,
                "0.02": 0.000040483604,
                "0.03": 0.000037966526,
                "0.04": 0.000036123608,
                "0.05": 0.000034383347
            }
        }
    },
    "1HZ25V": {
        "digits": 1,
        "precision": 2,
        "accu": {
            "barrier": {
                "0.01": 0.000108284915,
                "0.02": 0.000101209011,
                "0.03": 0.000094916316,
                "0.04": 0.00009030902,
                "0.05": 0.000085958368
            }
        }
    },
    "1HZ50V": {
        "digits": 1,
        "precision": 2,
        "accu": {
            "barrier": {
                "0.01": 0.000216569832,
                "0.02": 0.000202418022,
                "0.03": 0.000189832631,
                "0.04": 0.000180618039,
                "0.05": 0.000171916735
            }
        }
    },
    "1HZ75V": {
        "digits": 1,
        "precision": 2,
        "accu": {
            "barrier": {
                "0.01": 0.000324854752,
                "0.02": 0.000303627035,
                "0.03": 0.000284748947,
                "0.04": 0.000270927058,
                "0.05": 0.000257875102
            }
        }
    },
    "1HZ100V": {
        "digits": 1,
        "precision": 2,
        "accu": {
            "barrier": {
                "0.01": 0.000433139675,
                "0.02": 0.000404836049,
                "0.03": 0.000379665263,
                "0.04": 0.000361236077,
                "0.05": 0.000343833468
            }
        }
    },
    "1HZ150V": {
        "digits": 1,
        "precision": 2
    },
    "1HZ250V": {
        "digits": 1,
        "precision": 2
    }
}

In [1405]:
BO = {
    "turbos": {
        "no_barriers": {
          "R_10": 10, "R_25": 10, "R_50": 10, "R_75": 10, "R_100": 10, "1HZ10V": 10, "1HZ25V": 10, "1HZ50V": 10, "1HZ75V": 10, "1HZ100V": 10, "1HZ150V": 10, "1HZ250V": 10
        },
        "min_dist_from_spot": {
          "R_10": 3, "R_25": 3, "R_50": 3, "R_75": 3, "R_100": 3, "1HZ10V": 3, "1HZ25V": 3, "1HZ50V": 3, "1HZ75V": 3, "1HZ100V": 3, "1HZ150V": 3, "1HZ250V": 3
        },
        "T_min": {
          "R_10": 300, "R_25": 300, "R_50": 300, "R_75": 300, "R_100": 300, "1HZ10V": 300, "1HZ25V": 300, "1HZ50V": 300, "1HZ75V": 300, "1HZ100V": 300, "1HZ150V": 300, "1HZ250V": 300
        },
        "T_max": {
          "R_10": 1000000, "R_25": 1000000, "R_50": 1000000, "R_75": 1000000, "R_100": 1000000, "1HZ10V": 1000000, "1HZ25V": 1000000, "1HZ50V": 1000000, "1HZ75V": 1000000, "1HZ100V": 1000000, "1HZ150V": 1000000, "1HZ250V": 1000000
        },
        "ip": {
          "R_10": 0.2, "R_25": 0.2, "R_50": 0.2, "R_75": 0.2, "R_100": 0.2, "1HZ10V": 0.2, "1HZ25V": 0.2, "1HZ50V": 0.2, "1HZ75V": 0.2, "1HZ100V": 0.2, "1HZ150V": 0.2, "1HZ250V": 0.2
        },
        "tick_comm_up_t": {
          "R_10": 1.1, "R_25": 1.1, "R_50": 1.1, "R_75": 1.35, "R_100": 1.1, "1HZ10V": 1.1, "1HZ25V": 1.1, "1HZ50V": 1.1, "1HZ75V": 1.1, "1HZ100V": 1.1, "1HZ150V": 1.1, "1HZ250V": 1.1
        },
        "tick_comm_up_i": {
          "R_10": 1.08, "R_25": 1.08, "R_50": 1.08, "R_75": 1.33, "R_100": 1.08, "1HZ10V": 1.08, "1HZ25V": 1.08, "1HZ50V": 1.08, "1HZ75V": 1.08, "1HZ100V": 1.08, "1HZ150V": 1.08, "1HZ250V": 1.08
        },
        "tick_comm_up_d": {
          "R_10": 1.05, "R_25": 1.05, "R_50": 1.05, "R_75": 1.3, "R_100": 1.05, "1HZ10V": 1.05, "1HZ25V": 1.05, "1HZ50V": 1.05, "1HZ75V": 1.05, "1HZ100V": 1.05, "1HZ150V": 1.05, "1HZ250V": 1.05
        },
        "tick_comm_down_t": {
          "R_10": 1, "R_25": 1, "R_50": 1, "R_75": 1.35, "R_100": 1, "1HZ10V": 1.1, "1HZ25V": 1.1, "1HZ50V": 1.1, "1HZ75V": 1.1, "1HZ100V": 1.1, "1HZ150V": 1.1, "1HZ250V": 1.1
        },
        "tick_comm_down_i": {
          "R_10": 1.08, "R_25": 1.08, "R_50": 1.08, "R_75": 1.33, "R_100": 1.08, "1HZ10V": 1.08, "1HZ25V": 1.08, "1HZ50V": 1.08, "1HZ75V": 1.08, "1HZ100V": 1.08, "1HZ150V": 1.08, "1HZ250V": 1.08
        },
        "tick_comm_down_d": {
          "R_10": 1.05, "R_25": 1.05, "R_50": 1.05, "R_75": 1.30, "R_100": 1.05, "1HZ10V": 1.05, "1HZ25V": 1.05, "1HZ50V": 1.05, "1HZ75V": 1.05, "1HZ100V": 1.05, "1HZ150V": 1.05, "1HZ250V": 1.05
        },
        "max_mul": {
          "R_10": 1000, "R_25": 500, "R_50": 200, "R_75": 150, "R_100": 100, "1HZ10V": 1000, "1HZ25V": 500, "1HZ50V": 200, "1HZ75V": 150, "1HZ100V": 100,"1HZ150V": 5, "1HZ250V": 5
        },
        "min_mul": {
          "R_10": 100, "R_25": 50, "R_50": 20, "R_75": 15, "R_100": 10, "1HZ10V": 100, "1HZ25V": 50, "1HZ50V": 20, "1HZ75V": 15, "1HZ100V": 10, "1HZ150V": 1, "1HZ250V": 1
        },
        "max_open_pos": {
          "R_10": 3, "R_25": 3, "R_50": 3, "R_75": 3, "R_100": 3, "1HZ10V": 3, "1HZ25V": 3, "1HZ50V": 3, "1HZ75V": 3, "1HZ100V": 3, "1HZ150V": 3, "1HZ250V": 3
        }
    },
    "vanillas": {
        "max_stake": {
            "USD": 2000, "BTC": 0.03
        },
        "vol_markup": 0.03,
        "spread_spot": {
            'R_10': 0.169, 'R_25': 0.129, 'R_50': 0.024, 'R_75': 120, 'R_100': 0.52, 
            '1HZ10V': 0.21, '1HZ25V': 27.03, '1HZ50V': 52.6, '1HZ75V': 1.96, '1HZ100V': 0.49, '1HZ150V': 2.26, '1HZ250V': 76.83
        },
        "delta_config": [0.1,0.25,0.5,0.75,0.9],
        "bs_markup": 0,
        "max_strike_price_choices": {
            "intraday": 5, "daily": 11
        },
        "min_ppp": {},
        "max_ppp": {},
        "max_daily_volume": {6000},
        "max_daily_pnl": 3000,
    },
    "accumulators": {
        "max_stake": {
            "USD": 500, "BTC": 0.00281360
        },
        "max_payout": 4000,
        "max_duration": {
            "growth_rate_0.01": 230, "growth_rate_0.02": 115, "growth_rate_0.03": 75,"growth_rate_0.04": 55,"growth_rate_0.05": 45
        },
        "growth_rate": [0.01,0.02,0.03,0.04,0.05],    
        "max_daily_volume": {20000},    
    }
    
}

In [1406]:
def insert_dict (item, _dict) -> dict:
    try: 
        item['mv'].update(_dict)
    except Exception as e:
        item['mv'] = {}
        item['mv'].update(_dict)
    return item

In [1445]:
class Product:
    """
    A base class to represent a product for trading contracts through DerivAPI via WebSocket.

    Attributes:
        api_token (str): API token for authentication.
        stake (int): The amount to stake in each trade.
        symbol (str): The symbol of the product.
        messages (list): List to store messages received from the WebSocket.
        endpoint (str): The WebSocket endpoint.
        retry (int): Counter for retry attempts when sending/receiving messages.
        contract_ids (list): List of contract IDs.
        ws (WebSocket): WebSocket connection object.

    Methods:
        __setattr__(key, value): Sets an attribute and creates a WebSocket connection if 'endpoint' is set.
        set_attributes(**kwargs): Sets multiple attributes from keyword arguments.
        proposal(): Generates a proposal based on the current proposal type.
        send(): Sends a proposal to the WebSocket and handles the response.
        send_message(message): Sends a user-defined JSON message to the WebSocket.
        stream(): Streams data from the WebSocket.
        price_proposal(buy=False, **kwargs): Gets a price proposal.
        buy_contract(random=False, **kwargs): Attempts to buy a contract.
        contract_proposal(subscribe=0, contract_id=None): Gets contract details and optionally subscribes to updates.
        _check_connection(): Checks the WebSocket connection and re-opens if closed.
        _check_closed(): Checks if a contract is closed.
        sell_contract(contract_id=None): Attempts to sell a contract.
        create_ws(): Creates a new WebSocket connection.
        authorise(): Authorises the WebSocket connection.
        recv_msg_handler(): Handles messages received from the WebSocket.
    """
    
    def __init__(self, api_token=0, stake=10, symbol = '1HZ100V', **kwargs):
        self.messages = []
        self.endpoint = "wss://blue.derivws.com/websockets/v3?app_id=16929"
        self.retry = 0
        self.contract_ids = []
        for key, value in kwargs.items():
            setattr(self, key, value)
        # Shared attributes
        self.api_token = api_token
        self.stake = stake
        self.symbol = symbol
        self.ws = websocket.create_connection(self.endpoint)
        self.stats = {}

    
    # Set attributes from kwargs
    def __setattr__(self, key: str, value) -> None:
        super().__setattr__(key, value)
        if key == 'endpoint':
            self.create_ws()

    def set_attributes(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    # Proposal generator
    def proposal(self):
        proposal = {}
        if self.proposal_type == 'buy_contract':
            proposal = {
                "buy": self.proposal_id,
                "price": self.stake
            }
        elif self.proposal_type == 'contract_proposal':
            proposal = {
                "proposal_open_contract": 1,
                "contract_id": self.contract_id
            }
            if self.subscribe == 1:
                proposal["subscribe"] = 1
        elif self.proposal_type == 'sell_contract':
            proposal = {
                "sell": self.contract_id,  
                "price": 0
            }
        # To ensure this class isn't used to do price proposals without a defined contract type
        elif self.proposal_type == 'price_proposal':
            raise NotImplementedError('Use a Subclass to use this method')
                
        return json.dumps(proposal)

    # Send json to ws
    def send(self):
        self._check_connection()
        self.ws.send(self.proposal())
        return self.recv_msg_handler()
        
    # Send use input json to ws
    def send_message(self, message) -> dict:
        self._check_connection()
        self.ws.send(json.dumps(message))
        try:
            temp = json.loads(self.ws.recv())
        except Exception as e:
            temp = self.send_message(message)
        self.last_msg = temp
        self.messages.append(self.last_msg)
        return temp
    
    # Stream from ws
    def stream(self):
        self._check_connection()
        self.ws.send(self.proposal())
        self.recv_msg_handler()
        if self._check_closed():
            print('Contract cannot be sold')
            self.subscribe = 0
            self.send()
        else:
            self.subscription_id = self.last_msg['subscription']['id']
            try:
                while True:
                    clear_output(wait = True)
                    self.recv_msg_handler()
                    display(self.last_msg)
                    print('Keyboard Interrupt to stop streaming')
                    if self._check_closed():
                        break
            except KeyboardInterrupt:
                pass
            self.ws.send(json.dumps({"forget": self.subscription_id}))
            self.recv_msg_handler()
        return 1
        
    # Get price proposal 
    def price_proposal(self, buy=False, **kwargs):
        self.set_attributes(**kwargs)
        self.proposal_type = 'price_proposal'
        success = self.send()
        if not buy and success:
            display(self.last_msg)
        return success
    
    # Attempt to buy contract
    def buy_contract(self, random=False, runs=1, **kwargs) -> None:
        i = 0
        while i < runs:
            i += 1
            if random or runs > 1:
                self.randomise_attributes()
            self.set_attributes(**kwargs)
            if self.price_proposal(buy=True):
                self.proposal_id = self.last_msg['proposal']['id']
                self.proposal_type = 'buy_contract'
                if self.send():
                    self.contract_ids.append(self.last_msg['buy']['contract_id'])
                    clear_output(wait=True)
                    print(i)
                    self.contract_proposal()
        
    # Get contract proposal 
    def contract_proposal(self, subscribe = 0, contract_id = None):
        if len(self.contract_ids) == 0:
            print("No Contracts")
        else:
            if contract_id == None:
                self.contract_id = self.contract_ids[-1]
            elif contract_id not in self.contract_ids:
                print('ID not available. Using last contract ID')
                self.contract_id = self.contract_ids[-1]
            else:
                self.contract_id = contract_id
            self.proposal_type = 'contract_proposal'
            self.subscribe = subscribe
            if self.subscribe == 0:
                self.send()
            elif self.subscribe == 1:
                self.stream()
            display(self.last_msg)
            
    # Get all proposal_open_contracts from self.contract_ids
    def get_all_poc(self):
        for temp_id in self.contract_ids:
            clear_output(wait=True)
            self.contract_proposal(contract_id=temp_id)
            
    def tick_history(self, symbol=False, count=1, end='latest'):
        temp_msg = {
            "ticks_history": symbol if symbol else self.symbol,
            "adjust_start_time": 1,
            "count": count,
            "end": end,
            "start": 1,
            "style": "ticks"
        }
        return self.send_message(temp_msg)
        
    def _check_connection(self):
        try:
            # Send a ping message
            self.ws.send(json.dumps({"ping": 1}))
            # Receive a response
            self.ws.recv()
        except Exception as e:
            print('Connection Closed, Reconnecting')
            time.sleep(0.2)
            clear_output(wait=True)
            self.create_ws()
            
    def _check_closed(self):
        if self.last_msg['proposal_open_contract']['is_expired'] or self.last_msg['proposal_open_contract']['is_sold']:
            return 1
        else: 
            return 0
            
    # Attempt to sell contract
    def sell_contract(self, contract_id = None):
        if contract_id == None:
            self.contract_id = self.contract_ids[-1]
        elif contract_id not in self.contract_ids:
            print('ID not available. Using last contract ID')
            self.contract_id = self.contract_ids[-1]
        else:
            self.contract_id = contract_id
        self.proposal_type = 'sell_contract'
        if self.send():
            display(self.last_msg)
            self.contract_proposal()
        else:
            print('Contract cannot be sold. Proposal below:')
            self.contract_proposal()
            
    def sell_all_contracts(self):
        for temp_id in self.contract_ids:
            clear_output(wait=True)
            self.sell_contract(contract_id=temp_id)
    
    def create_ws(self):
        self.ws = 0
        self.ws = websocket.create_connection(self.endpoint)

    def authorise(self):
        self._check_connection()
        self.ws.send(json.dumps({"authorize": self.api_token}))
        if self.recv_msg_handler():
            return 1
        
    # Receive handling 
    def recv_msg_handler(self):
        temp = self.ws.recv()
        try:
            self.last_msg = json.loads(temp)
        except Exception as e:
            self.retry = 0
            return self.send()
        self.messages.append(self.last_msg)
#         display(self.last_msg)
        if 'error' in self.last_msg:
            self.retry += 1
            error = self.last_msg['error']
            if error['code'] in ['InvalidToken', 'OpenPositionLimitExceeded', 'InvalidSellContractProposal']:
                display(error)
                self.retry = 0
                return 0            
            elif error['code'] == 'OfferingsValidationError' or error['code'] == 'RateLimit' or error['message'] == 'Resale of this contract is not offered.' :
                    if error['message'] == 'Number of ticks must be between 5 and 10.':
                        self.duration = np.random.randint(5,11)
                    elif error['code'] == 'RateLimit':
                        self.create_ws()
                    elif error['message'] == 'Trading is not offered for this asset.':
                        self.symbol = 'R_100'        
                    else:
                        display(error)
                        self.retry = 0
                        return 0
            elif error['code'] == 'AuthorizationRequired':
                self.authorise()
            elif error['code'] == 'ContractBuyValidationError':
                if 'message' in error.keys() and error['message'] == 'This trade is temporarily unavailable.':
                    print(error['message'])
                    self.retry = 0
                    return 0
                if error['details']['field'] == 'barrier':
                    try: 
                        self.strike = random.choice(error['details']['barrier_choices'])
                    except Exception as e:
                        self.strike = '1000'
                elif error['details']['field'] == 'amount':
                    display(self.last_msg)
                    self.stake = error['details']['min_stake']
                elif error['details']['field'] == 'payout_per_point':
                    self.payout_per_point = random.choice(error['details']['payout_per_point_choices'])
            elif error['code'] == 'InvalidContractProposal':
                self.buy_contract()
            elif error['code'] == 'ContractCreationFailure':
                self.randomise_attributes()
            else:
                print('Undefined Error')
                display(error)
            if self.retry > 8: 
                print('max retries made')
            elif self.retry > 0: 
                return self.send()
        else:
            self.retry = 0
            return 1

In [1408]:
def accu_barriers(prop) -> tuple:
    """ Determines if higher, lower and barrier distance of current stop are accurate for an accumulator contract"""
    proposal, echo_req, contract_details = prop['proposal'], prop['echo_req'], proposal['contract_details']
    high_bar, low_bar, bar_dist = \
        (round(proposal['spot'] * (1+symbols[echo_req['symbol']]['accu']['barrier'][str(echo_req['growth_rate'])]),symbols[echo_req['symbol']]['precision']+1)), \
        (round(proposal['spot'] * (1-symbols[echo_req['symbol']]['accu']['barrier'][str(echo_req['growth_rate'])]),symbols[echo_req['symbol']]['precision']+1)), \
        (round(proposal['spot'] * (symbols[echo_req['symbol']]['accu']['barrier'][str(echo_req['growth_rate'])]),symbols[echo_req['symbol']]['precision']+1))
    pass_fail =\
    abs(float(contract_details['high_barrier']) - high_bar) <= 1/10**symbols[echo_req['symbol']]['precision'] and \
    abs(float(contract_details['low_barrier']) - low_bar) <= 1/10**symbols[echo_req['symbol']]['precision'] and \
    abs(float(contract_details['barrier_spot_distance']) - bar_dist) <= 1/10**symbols[echo_req['symbol']]['precision']
    return high_bar, low_bar, bar_dist, pass_fail


In [1409]:
# Accumulator Class
class Accumulator(Product):
    """
    A class to represent an Accumulator product, inheriting from the Product class.

    Attributes:
        species (str): The type of the product, 'accumulator'.
        contract_type (str): The type of contract, default is 'ACCU'.
        growth_rate (float): The growth rate of the accumulator, default is 0.05.

    Methods:
        randomise_attributes(): Randomises the attributes of the Accumulator instance.
        proposal(): Generates a proposal specific to the Accumulator product.
        check_barrier(runs=0, show=False): Checks barrier conditions for the given number of runs.
    """
    
    species = "accumulator"
    
    def __init__(self, api_token=0, stake=10, symbol='1HZ100V', **kwargs):
        super().__init__(api_token, stake, symbol, **kwargs)
        self.contract_type = 'ACCU'
        self.growth_rate = kwargs.get('growth_rate', 0.05)

    # randomise attributes 
    def randomise_attributes(self):
        self.symbol = random.choice(list(symbols.keys()))
        self.stake = round(np.random.uniform(1,BO['accumulators']['max_stake']['USD']),2)
        self.growth_rate = random.choice([0.01,0.02,0.03,0.04,0.05])

        # Proposal generator
    def proposal(self):
        proposal = {}
        if self.proposal_type == 'price_proposal':
            proposal = {
                "proposal": 1,
                "amount": self.stake,
                "basis": "stake",
                "contract_type": self.contract_type,
                "currency": "USD",
                "symbol": self.symbol,
                'growth_rate' : self.growth_rate
            }
        else:
            return super().proposal()
        return json.dumps(proposal)
    
        # check ?
    def check_barrier(self, runs=0, show = False, **kwargs):
        for i in range(runs):
            clear_output(wait = True)
            self.randomise_attributes()
            self.set_attributes(**kwargs)
            self.price_proposal()
        clear_output(wait = True)
        self.accu_barrier_stats = {'stat': 'accu_barrier_stats', 'cases': 0, 'pass': 0, 'fail': 0}        
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal' and 'error' not in item:
                temp_dict = {}
                temp_dict['high_barrier'], temp_dict['low_barrier'], temp_dict['barrier_spot_distance'], temp_dict['barrier_check'] = accu_barriers(item)
                item = insert_dict(item, temp_dict)
                self.accu_barrier_stats['cases'] += 1
                if item['mv']['barrier_check']: 
                    self.accu_barrier_stats['pass'] += 1
                else: 
                    self.accu_barrier_stats['fail'] += 1                
                if not item['mv']['barrier_check'] and show:
                    display(item)
                     
        display(self.accu_barrier_stats)
        
    # check ask price for all proposals in messages and if specified, additional random contracts (WIP)                      
    def bid_check(self, runs=0, show = False):
        for i in range(runs):
            clear_output(wait = True)
            self.randomise_attributes()
            self.price_proposal()
        clear_output(wait = True)
        self.vanilla_ask_ppp_stats = {'stat': 'vanilla_ask_ppp_stats', 'cases': 0, 'pass': 0, 'fail': 0}        
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal' and 'error' not in item:
                ask_ppp_pass_fail = float(item['proposal']['display_number_of_contracts']) == item['mv']['ask_ppp']
                item['mv']['ask_ppp_check'] = ask_ppp_pass_fail
                self.vanilla_ask_ppp_stats['cases'] += 1
                if ask_ppp_pass_fail: 
                    self.vanilla_ask_ppp_stats['pass'] += 1
                else: 
                    self.vanilla_ask_ppp_stats['fail'] += 1                    
                if not ask_ppp_pass_fail and show:
                    display(item)
        display(self.vanilla_ask_ppp_stats)

In [1410]:
def vanilla_barriers(prop) -> list:
    """Calculate expected barrier choices for vanillas based on given json"""
    spot = prop['proposal']['spot']
    symbol = prop['echo_req']['symbol']
    duration = [prop['proposal']['date_start'], prop['proposal']['date_expiry']]
    duration_unit = prop['echo_req']['duration_unit']
    digits = symbols[symbol]['digits']
    sigma = int(re.search(r'(\d+)[^\d]*$', symbol).group(1))/100
    n_digits = round(-np.log10(spot * sigma * np.sqrt(60 / 365 / 86400)), 0) - 1
    spot = round(spot / 10 ** -n_digits) * 10 ** -n_digits
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400    
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if duration_unit != 'd':
        deltas = np.array(BO['vanillas']['delta_config'])
        real_strikes = spot * np.exp((sigma ** 2) / 2 * t - (sigma * np.sqrt(t) * norm.ppf(deltas)))
        displayed_barriers = np.round((real_strikes - spot) * 10 ** digits) / 10 ** digits
    else:
        temp_barriers = []
        displayed_barriers = []
        central_strike = round(spot / (10 ** -n_digits)) * (10 ** -n_digits)
        max_strike = np.floor((spot * np.exp((sigma ** 2) / 2 * t - (sigma * np.sqrt(t) * norm.ppf(0.1)))) / (10 ** -n_digits)) * (10 ** -n_digits)
        min_strike = np.ceil((spot * np.exp((sigma ** 2) / 2 * t - (sigma * np.sqrt(t) * norm.ppf(0.9)))) / (10 ** -n_digits)) * (10 ** -n_digits)
        strike_step_1 = round((central_strike - min_strike) / 5 / 10) * 10
        strike_step_2 = round((max_strike - central_strike) / 5 / 10) * 10         
        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1 * i)            
        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2 * i)            
        temp_barriers.append(max_strike)        
        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)
    return displayed_barriers

def vanilla_ppp_ask (prop) -> float:
    """Calculate expected payout per point (ppp) for a vanilla contract based on the given json"""    
    spot = prop['proposal']['spot']
    strike = float(prop['proposal']['contract_details']['barrier'])      
    number_of_contracts = prop['proposal']['display_number_of_contracts']
    api_ppp = float(prop['proposal']['display_number_of_contracts'])
    symbol = prop['echo_req']['symbol']
    r = 0 if 'pricing_args' not in prop['proposal'] else prop['proposal']['pricing_args']['discount_rate']
    contract_type = prop['echo_req']['contract_type']
    duration = [prop['proposal']['date_start'], prop['proposal']['date_expiry']]
    stake = prop['echo_req']['amount']
    vol_charge = 1 + BO['vanillas']['vol_markup']
    t = ( int(duration[1]) - int(duration[0]) ) / 365 / 86400  
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
    sigma = get_sigma(symbol)
    spot_spread = BO['vanillas']['spread_spot'][symbol]
    bs_markup = BO['vanillas']['bs_markup']        
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))
    if contract_type == "VANILLALONGCALL":
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2        
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))    
    digits_precision = number_of_contracts.split('.')
    digits_precision = 0 if len(digits_precision) < 2 else len(digits_precision[1])
    ppp = round(stake / ( bs_price + delta_charge + bs_markup ), digits_precision)
    return ppp, ppp == api_ppp

In [1411]:
# Vanilla Class
class Vanilla(Product):
    """
    A class to represent a Vanilla product, inheriting from the Product class.

    Attributes:
        contract_type (str): The type of contract, either 'VANILLALONGCALL' or 'VANILLALONGPUT'.
        strike (str): The strike price for the contract.
        duration (int): The duration of the contract.
        duration_unit (str): The unit of the duration ('m' for minutes, 'h' for hours, 'd' for days).

    Methods:
        randomise_attributes(): Randomises the attributes of the Vanilla instance.
        proposal(): Generates a proposal specific to the Vanilla product.
        check_barrier(runs=0, show=False): Validates the offered barriers in all proposals + for the given number of runs if defined.
        check_ask(runs=0, show=False): Validates the payout per point (ask price) in all proposals + for the given number of runs if defined.
        bid_check(runs=0, show=False): Validates the payout (bid price) in all proposals + for the given number of runs if defined.
    """
        
    species = "vanilla"
    
    def __init__(self, api_token=0, stake=10, symbol='1HZ100V', **kwargs):
        super().__init__(api_token, stake, symbol, **kwargs)
        self.species = "vanilla"
        self.strike = kwargs.get('strike', '+0.00')
        self.contract_type = 'VANILLALONGCALL' if 'c' in kwargs.get('contract_type', 'call') else 'VANILLALONGPUT'
        self.duration = kwargs.get('duration', 1)
        self.duration_unit = kwargs.get('duration_unit', 'm')
            
    # randomise attributes 
    def randomise_attributes(self):
        self.symbol = random.choice(list(symbols.keys()))
        self.duration_unit = random.choice(['m','h','d'])
        self.strike = '+0.00' if self.duration_unit != 'd' else '1000'
        self.contract_type = random.choice(['VANILLALONGCALL','VANILLALONGPUT'])
        self.duration = np.random.randint(1,61) if self.duration_unit == 'm' else\
                        np.random.randint(1,25) if self.duration_unit == 'h' else\
                        np.random.randint(1, (14 if self.symbol == '1HZ250V' else 40 if self.symbol == '1HZ150V' else 365)) 
        
    # Proposal generator
    def proposal(self):
        proposal = {}
        if self.proposal_type == 'price_proposal':
            proposal = {
                "proposal": 1,
                "amount": self.stake,
                "basis": "stake",
                "contract_type": self.contract_type,
                "currency": "USD",
                "symbol": self.symbol,
                'barrier' : self.strike,
                'duration' : self.duration,
                'duration_unit' : self.duration_unit
            }
        else:
            return super().proposal()
        return json.dumps(proposal)

    # check barriers for all proposals in messages and if specified, additional random contracts
    def check_barrier(self, runs=0, show = False, **kwargs):
        for i in range(runs):
            clear_output(wait = True)
            self.randomise_attributes()
            self.set_attributes(**kwargs)
            self.price_proposal()
        clear_output(wait = True)
        self.vanilla_barrier_stats = {'stat': 'vanilla_barrier_stats', 'cases': 0, 'pass': 0, 'fail': 0}
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal' and 'error' not in item:
                temp_dict = {'barrier_choices': vanilla_barriers(item)}
                item = insert_dict(item, temp_dict)    
                try:
                    barrier_pass_fail =  all([float(bar) for bar in item['proposal']['barrier_choices']] == item['mv']['barrier_choices'])
                except:
                    item['mv']['barrier_choices'] = sorted(item['mv']['barrier_choices'])
                    barrier_pass_fail = [float(bar) for bar in item['proposal']['barrier_choices']] == item['mv']['barrier_choices']
                item['mv']['barrier_check'] = barrier_pass_fail
                self.vanilla_barrier_stats['cases'] += 1
                if barrier_pass_fail: 
                    self.vanilla_barrier_stats['pass'] += 1
                else: 
                    self.vanilla_barrier_stats['fail'] += 1               
                if not barrier_pass_fail and show:
                    display(item)
        display(self.vanilla_barrier_stats)

    # check ask price for all proposals in messages and if specified, additional random contracts                      
    def check_ask(self, runs=0, show = False, **kwargs):
        for i in range(runs):
            clear_output(wait = True)
            self.randomise_attributes()
            self.set_attributes(**kwargs)
            self.price_proposal()
        clear_output(wait = True)
        self.vanilla_ask_ppp_stats = {'stat': 'vanilla_ask_ppp_stats', 'cases': 0, 'pass': 0, 'fail': 0}        
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal' and 'error' not in item:
                temp_dict = {}
                temp_dict['ask_ppp'], temp_dict['ask_ppp_check'] = vanilla_ppp_ask(item)                
                item = insert_dict(item, temp_dict)
                self.vanilla_ask_ppp_stats['cases'] += 1
                if item['mv']['ask_ppp_check']: 
                    self.vanilla_ask_ppp_stats['pass'] += 1
                else: 
                    self.vanilla_ask_ppp_stats['fail'] += 1                    
                if not item['mv']['ask_ppp_check'] and show:
                    display(item)
        display(self.vanilla_ask_ppp_stats)
                       
    # check ask price for all proposals in messages and if specified, additional random contracts                      
    def check_bid(self, runs=0, show=False):
        for i in range(runs):
            clear_output(wait = True)
            self.buy_contract(random=1)
        self.get_all_poc()
        clear_output(wait = True)
        self.vanilla_bid_stats = {'stat': 'vanilla_bid_stats', 'cases': 0, 'pass': 0, 'fail': 0}        
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal_open_contract' and 'error' not in item:
                ask_ppp_pass_fail = float(item['proposal']['display_number_of_contracts']) == item['mv']['ask_ppp']
                item['mv']['ask_ppp_check'] = ask_ppp_pass_fail
                self.vanilla_ask_ppp_stats['cases'] += 1
                if ask_ppp_pass_fail: 
                    self.vanilla_ask_ppp_stats['pass'] += 1
                else: 
                    self.vanilla_ask_ppp_stats['fail'] += 1                    
                if not ask_ppp_pass_fail and show:
                    display(item)
        display(self.vanilla_ask_ppp_stats)

In [1412]:
def get_sigma (string) -> float:
    """Returns sigma from given symbol (str)"""
    return int(re.search(r'(\d+)[^\d]*$', string).group(1))/100

def roundsf(string) -> float:
    """Rounds the given numeric string to retain only the first non-zero digit and replaces all following digits with zeros"""
    string = "{:.15f}".format(string)
    non_zero_index = re.search('[^0.]', string).start()        
    return float(string[:non_zero_index+1] + re.sub('[0-9]', '0', string[non_zero_index+1:]))

def roundup (num, prec) -> float:
    """Roundup a number to the specified precision"""
    return np.ceil(round((num * 10**prec),15)) / (10**prec)

def rounddown (num, prec) -> float:
    """Rounddown a number to the specified precision"""    
    return np.floor(round((num * 10**prec),15)) / (10**prec)

def turbo_ask (symbol, spot, barrier, duration_unit, phi) -> float:
    """Calculates the ask price for 1 turbo contract"""
    tick_comm = 'tick_comm_' + ('up' if phi == 1 else 'down')
    tick_comm += '_t' if  duration_unit == 't' else  '_d' if duration_unit == 'd' else '_i'
    sigma = int(re.search(r'(\d+)[^\d]*$', symbol).group(1))/100
    is_r = 2 if re.search(re.compile(r'R'), symbol) else 1
    if phi == 1:
        commup = BO['turbos'][tick_comm][symbol] * sigma * np.sqrt(is_r/365/86400)
        return spot * (1 + commup) - barrier
    else:
        commdown = BO['turbos'][tick_comm][symbol] * sigma * np.sqrt(is_r/365/86400)
        return barrier - spot * (1 - commdown) 

def turbo_barrier(prop, start_of_min_spot) -> float:
    """Calculate expected barrier from proposal"""
    phi = 1 if prop['echo_req']['contract_type'] == 'TURBOSLONG' else -1
    symbol = prop['echo_req']['symbol']
    spot = start_of_min_spot
    ppp = prop['echo_req']['payout_per_point']
    stake = prop['echo_req']['amount']
    tick_comm = 'tick_comm_' + ('up' if phi == 1 else 'down')
    tick_comm += '_t' if  prop['echo_req']['duration_unit'] == 't' else  '_d' if prop['echo_req']['duration_unit'] == 'd' else '_i'
    sigma = get_sigma(symbol)
    is_r = 2 if re.search(re.compile(r'R'), symbol) else 1
    comm = BO['turbos'][tick_comm][symbol] * sigma * np.sqrt(is_r/365/86400)
    if phi == 1:
        barrier = roundup(spot * comm - stake/ppp, symbols[symbol]['precision'])
    else:
        barrier = rounddown(stake/ppp - spot * comm, symbols[symbol]['precision'])
    return round(prop['proposal']['spot'] + barrier, symbols[symbol]['precision']) 
    
def turbo_ppps(prop, start_of_min_spot) -> tuple[list, bool]:
    """Calculates the expected offered payout_per_point choices from the given json"""
    stake = prop['echo_req']['amount']
    phi = 1 if 'LONG' in prop['echo_req']['contract_type'] else -1
    symbol = prop['echo_req']['symbol']
    spot = start_of_min_spot
    duration_unit = prop['echo_req']['duration_unit']
    sigma = int(re.search(r'(\d+)[^\d]*$', symbol).group(1))/100
    barrier_far = spot * (1 - phi * sigma * np.sqrt(BO['turbos']['T_max'][symbol]/365/86400))
    barrier_close = spot * (1 - phi * sigma * np.sqrt(BO['turbos']['T_min'][symbol]/365/86400))
    n_far = stake / turbo_ask(symbol, spot, barrier_far, duration_unit, phi)
    n_close = stake / turbo_ask(symbol, spot, barrier_close, duration_unit, phi)
    increment =  roundsf((n_close - n_far) * BO['turbos']['ip'][symbol])
    tn = roundup(n_far/increment,0) #tn is the smallests ppp
    t0 = rounddown(n_close/increment,0) #t0 is the largest
    ppp = [tn*increment]
    while ppp[-1] < t0*increment:
        ppp.append(round(ppp[-1]+increment,12))
    ppp = np.array(ppp)
    offered_pps = ppp[ppp <= round(t0*increment,12)]
    barrier = turbo_barrier(prop, spot)
    return offered_pps, barrier, sorted(offered_pps) == sorted(prop['proposal']['payout_choices']) and barrier == float(prop['proposal']['contract_details']['barrier'])

def turbo_closed (prop):
    '''
    payout, match_flag, close_flag 
    '''
    symbol = prop['proposal_open_contract']['underlying']
    is_r = 2 if re.search(re.compile(r'R'), symbol) else 1
    ctype = prop['proposal_open_contract']['contract_type']
    try:
        spot = prop['proposal_open_contract']['exit_tick'] 
    except Exception as e:
        spot = prop['proposal_open_contract']['current_spot']
    barrier = float(prop['proposal_open_contract']['barrier'])
    bid_price = prop['proposal_open_contract']['bid_price']
    
    ppp = float(prop['proposal_open_contract']['display_number_of_contracts'])
    shortcode = prop['proposal_open_contract']['shortcode']
    sigma = get_sigma(symbol)
    if 'sell_time' in prop['proposal_open_contract'] and prop['proposal_open_contract']['sell_time'] >= prop['proposal_open_contract']['expiry_time']:
        if (ctype == 'TURBOSLONG' and spot <= barrier) or (ctype == 'TURBOSSHORT' and spot >= barrier):
            payout, close_flag = 0, 'ko'
        else:
            payout, close_flag = abs(spot - barrier) * ppp, 'expired'
    else:
        comm = 'down_'if ctype == 'TURBOSLONG' else 'up_'
        comm += 't' if 'T' in shortcode[11:] else 'i' if prop['proposal_open_contract']['is_intraday'] else 'd'
        comm = 'tick_comm_' + comm
        comm = BO['turbos'][comm][symbol] * sigma * np.sqrt(is_r/365/86400)
        payout, close_flag = max( (spot*(1 - comm) - barrier) * ppp if ctype == 'TURBOSLONG' else (barrier - spot*(1 + comm)) * ppp, 0), 'sold' if prop['proposal_open_contract']['is_sold'] else 'bid'
    match_flag = np.round(payout,2) == bid_price
    return payout, match_flag, close_flag

#not in use
def turbo_bid (symbol, spot, barrier, duration_unit, phi) -> float:
    """Calculates the bid price for 1 turbo contract from the given json"""
    tick_comm = 'tick_comm_' + ('down' if phi == 1 else 'up')
    tick_comm += '_t' if  duration_unit == 't' else  '_d' if duration_unit == 'd' else '_i'
    sigma = int(re.search(r'(\d+)[^\d]*$', symbol).group(1))/100
    is_r = 2 if re.search(re.compile(r'R'), symbol) else 1
    if phi == 1:
        commdown = BO['turbos'][tick_comm][symbol] * sigma * np.sqrt(is_r/365/86400)
        return max(spot * (1 - commdown) - barrier, 0)
    else:
        commup = BO['turbos'][tick_comm][symbol] * sigma * np.sqrt(is_r/365/86400)
        return max(barrier - spot * (1 + commup), 0)
    
def turbo_barriers(prop):
    spot = spot
    symbol = symbol
    duration = [prop['proposal']['date_start'], prop['proposal']['date_expiry']]
    duration_unit = prop['echo_req']['duration_unit']
    digits = symbols[symbol]['digits']
    sigma = int(re.search(r'(\d+)[^\d]*$', symbol).group(1))/100
    n_digits = round(-np.log10(spot * sigma * np.sqrt(60 / 365 / 86400)), 0) - 1
    spot = round(spot / 10 ** -n_digits) * 10 ** -n_digits
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400    
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if duration_unit != 'd':
        deltas = np.array(BO['turbos']['delta_config'])
        real_strikes = spot * np.exp((sigma ** 2) / 2 * t - (sigma * np.sqrt(t) * norm.ppf(deltas)))
        displayed_barriers = np.where(deltas > 0.5,\
                                      np.round((real_strikes - spot) * 10 ** digits) / 10 ** digits, \
                                      np.round((real_strikes - spot) * 10 ** digits) / 10 ** digits)
    else:
        temp_barriers = []
        displayed_barriers = []
        central_strike = round(spot / (10 ** -n_digits)) * (10 ** -n_digits)
        max_strike = np.floor((spot * np.exp((sigma ** 2) / 2 * t - (sigma * np.sqrt(t) * norm.ppf(0.1)))) / (10 ** -n_digits)) * (10 ** -n_digits)
        min_strike = np.ceil((spot * np.exp((sigma ** 2) / 2 * t - (sigma * np.sqrt(t) * norm.ppf(0.9)))) / (10 ** -n_digits)) * (10 ** -n_digits)
        strike_step_1 = round((central_strike - min_strike) / 5 / 10) * 10
        strike_step_2 = round((max_strike - central_strike) / 5 / 10) * 10         
        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1 * i)            
        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2 * i)            
        temp_barriers.append(max_strike)        
        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)
    return displayed_barriers

#not in use
def turbo_ppp_ask (prop) -> float:
    """Calculates the expected offered payout_per_point choices from the given json"""
    
    spot = spot
    strike = float(prop['proposal']['contract_details']['barrier'])      
    number_of_contracts = prop['proposal']['display_number_of_contracts']
    symbol = symbol
    r = 0 if 'pricing_args' not in prop['proposal'] else prop['proposal']['pricing_args']['discount_rate']
    contract_type = prop['echo_req']['contract_type']
    duration = [prop['proposal']['date_start'], prop['proposal']['date_expiry']]
    stake = stake
    vol_charge = 1 + BO['turbos']['vol_markup']
    t = ( int(duration[1]) - int(duration[0]) ) / 365 / 86400  
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
    r = 0 if 'pricing_args' not in prop['proposal'] else prop['proposal']['pricing_args']['discount_rate']
    contract_type = prop['echo_req']['contract_type']
    sigma = int(re.search(r'(\d+)[^\d]*$', symbol).group(1))/100
    spot_spread = BO['turbos']['spread_spot'][symbol]
    bs_markup = BO['turbos']['bs_markup']        
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))
    if contract_type == "TURBOSLONG":
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2        
    elif  contract_type == "TURBOSSHORT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))    
    digits_precision = number_of_contracts.split('.')
    digits_precision = 0 if len(digits_precision) < 2 else len(digits_precision[1])
    return round(stake / ( bs_price + delta_charge + bs_markup ), digits_precision)

In [1413]:
class Turbo(Product):
    """
    A class to represent a Turbo product, inheriting from the Product class.

    Attributes:
        species (str): The type of the product, 'turbo'.
        strike (str): The strike price for the contract, default is '+10.00'.
        payout_per_point (float): The payout per point, default is 2.
        contract_type (str): The type of contract, either 'TURBOSLONG' or 'TURBOSSHORT'.
        duration (int): The duration of the contract, default is 5.
        duration_unit (str): The unit of the duration ('t' for ticks, 's' for seconds, 'm' for minutes, 'h' for hours, 'd' for days).

    Methods:
        randomise_attributes(): Randomises the attributes of the Turbo instance.
        proposal(): Generates a proposal specific to the Turbo product.
    """
    
    species = "turbo"
    
    def __init__(self, api_token=0, stake=10, symbol='1HZ100V', **kwargs):
        super().__init__(api_token, stake, symbol, **kwargs)
        self.strike = kwargs.get('strike', '+10.00')
        self.payout_per_point = kwargs.get('payout_per_point', 2)
        self.contract_type = 'TURBOSLONG' if 'long' in kwargs.get('contract_type', 'TURBOSLONG') else 'TURBOSSHORT'
        self.duration = kwargs.get('duration', 5)
        self.duration_unit = kwargs.get('duration_unit', 'm')
            
    # randomise attributes 
    def randomise_attributes(self):
        self.symbol = random.choice(list(symbols.keys()))
        self.duration_unit = random.choice(['t','s','m','h','d'])
        self.strike = '+10.00'
        self.payout_per_point = 2.00001
        self.contract_type = random.choice(['TURBOSLONG','TURBOSSHORT'])
        self.duration = np.random.randint(5, 11) if self.duration_unit == 't' else\
                        np.random.randint(15, 86401) if self.duration_unit == 's' else\
                        np.random.randint(1, 1441) if self.duration_unit == 'm' else\
                        np.random.randint(1, 25) if self.duration_unit == 'h' else\
                        np.random.randint(1, 365) 
        
    # Proposal generator
    def proposal(self):
        proposal = {}
        if self.proposal_type == 'price_proposal':
            proposal = {
                "proposal": 1,
                "amount": self.stake,
                "basis": "stake",
                "contract_type": self.contract_type,
                "currency": "USD",
                "symbol": self.symbol,
#                 'barrier' : self.strike,
                'payout_per_point' : self.payout_per_point,
                'duration' : self.duration,
                'duration_unit' : self.duration_unit
            }
        else:
            return super().proposal()
        return json.dumps(proposal)

    def get_start_of_min_tick(self, prop):
        cur_time = prop['proposal']['date_start']
        start_of_min = cur_time - cur_time%60
        time.sleep(0.5)
        return self.tick_history(symbol=prop['echo_req']['symbol'], end=start_of_min)['history']['prices'][-1]
            
    def check_ppps(self, runs=0, show = False, **kwargs):
        for i in range(runs):
            time.sleep(0.71)
            clear_output(wait = True)
            self.randomise_attributes()
            self.set_attributes(**kwargs)
            self.price_proposal()
        clear_output(wait = True)
        self.stats['ppp_choices_check'] = {'cases': 0, 'pass': 0, 'fail': 0}
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal' and 'error' not in item:
                if 'mv' not in item or ('mv' in item and 'ppp_choices_check' not in item['mv']):
                    display(item)
                    temp_dict = {}
                    start_of_min_spot = self.get_start_of_min_tick(item)
                    temp_dict['payout_choices'], temp_dict['barrier'], temp_dict['ppp_choices_check'] = turbo_ppps(item, start_of_min_spot)
                    item = insert_dict(item, temp_dict)
                self.stats['ppp_choices_check']['cases'] += 1
                if item['mv']['ppp_choices_check']: 
                    self.stats['ppp_choices_check']['pass'] += 1
                else: 
                    self.stats['ppp_choices_check']['fail'] += 1
                clear_output(wait=True)
        if show:
            for item in self.messages:
                if 'msg_type' in item and item['msg_type'] == 'proposal' and 'error' not in item and not item['mv']['ppp_choices_check']:
                    display(item)                     
        display(self.stats)
        
    def check_payout(self, runs=0, show = False, **kwargs):
        for i in range(runs):
            clear_output(wait = True)
            self.buy_contract(random=1, **kwargs)
        self.get_all_poc()
        clear_output(wait = True)
        self.stats['payout_check'] = {
            'total': {'cases': 0, 'pass': 0, 'fail': 0},
            'ko': {'cases': 0, 'pass': 0, 'fail': 0},
            'expired': {'cases': 0, 'pass': 0, 'fail': 0},
            'sold': {'cases': 0, 'pass': 0, 'fail': 0},
            'bid': {'cases': 0, 'pass': 0, 'fail': 0}
        }
        for item in self.messages:
            if 'msg_type' in item and item['msg_type'] == 'proposal_open_contract' and 'error' not in item:
                temp_dict = {}
                temp_dict['payout'], temp_dict['payout_check'], temp_dict['close_flag'] = turbo_closed(item) 
                item = insert_dict(item, temp_dict)
                self.stats['payout_check'][temp_dict['close_flag']]['cases'] += 1
                self.stats['payout_check']['total']['cases'] += 1
                if item['mv']['payout_check']: 
                    self.stats['payout_check'][temp_dict['close_flag']]['pass'] += 1
                    self.stats['payout_check']['total']['pass'] += 1
                else: 
                    self.stats['payout_check'][temp_dict['close_flag']]['fail'] += 1  
                    self.stats['payout_check']['total']['fail'] += 1
                if not item['mv']['payout_check'] and show:
                    display(item) 
        display(self.stats)

In [1414]:
tt = Turbo(endpoint = 'wss://qa69.deriv.dev/websockets/v3?l=EN&app_id=16303&brand=binary', api_token = '')